## RAG에서의 문맥 강화 검색
검색 증강 생성(RAG)은 외부 소스에서 관련 지식을 검색하여 AI 응답을 향상시킵니다. 기존 검색 방법은 분리된 텍스트 청크를 반환하여 불완전한 답변을 초래할 수 있습니다.
이 문제를 해결하기 위해, 검색된 정보가 더 나은 일관성을 위해 주변 청크를 포함하도록 보장하는 **문맥 강화 검색(Context-Enriched Retrieval)**을 소개합니다.
단순히 가장 유사한 청크 하나만 가져오는 대신, 해당 청크의 앞뒤에 있는 문맥적으로 관련된 청크들을 함께 가져와 언어 모델에 제공함으로써, 언어 모델이 더 풍부한 정보를 바탕으로 완전하고 정확한 답변을 생성하도록 돕습니다.

이 노트북의 단계:
- **데이터 수집**: PDF에서 텍스트를 추출합니다.
- **문맥을 포함한 청킹**: 텍스트를 문맥을 보존하기 위해 중첩되는 청크로 분할합니다.
- **임베딩 생성**: 텍스트 청크를 숫자 표현으로 변환합니다.
- **문맥 인식 검색**: 더 나은 완전성을 위해 관련 청크를 주변 청크와 함께 검색합니다.
- **응답 생성**: 검색된 문맥을 기반으로 언어 모델을 사용하여 응답을 생성합니다.
- **평가**: 모델의 응답 정확도를 평가합니다.

## 환경 설정
필요한 라이브러리를 가져오는 것으로 시작합니다.

In [1]:
import fitz # PyMuPDF 라이브러리
import os
import numpy as np
import json
from openai import OpenAI

## PDF 파일에서 텍스트 추출
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 여기서는 PyMuPDF 라이브러리를 사용하여 PDF 파일에서 텍스트를 추출합니다.

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출하고 처음 `num_chars`개의 문자를 출력합니다.

    Args:
    pdf_path (str): PDF 파일 경로.

    Returns:
    str: PDF에서 추출된 텍스트.
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 추출된 텍스트를 저장할 빈 문자열 초기화

    # PDF의 각 페이지 반복
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 all_text 문자열에 추가

    return all_text  # 추출된 텍스트 반환

## 추출된 텍스트 청킹
추출된 텍스트가 있으면 검색 정확도를 높이기 위해 더 작고 중첩되는 청크로 나눕니다.

In [3]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n개의 문자로 된 세그먼트로 나누고 중첩을 허용합니다.

    Args:
    text (str): 청킹할 텍스트.
    n (int): 각 청크의 문자 수.
    overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
    List[str]: 텍스트 청크 목록.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    
    # (n - overlap) 크기의 단계로 텍스트 반복
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트 청크를 청크 목록에 추가
        chunks.append(text[i:i + n])

    return chunks  # 텍스트 청크 목록 반환

## OpenAI API 클라이언트 설정
임베딩과 응답을 생성하기 위해 OpenAI 클라이언트를 초기화합니다.

In [ ]:
# 기본 URL과 API 키로 OpenAI 클라이언트 초기화
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 환경 변수에서 API 키 검색
)

## PDF 파일에서 텍스트 추출 및 청킹
이제 PDF를 로드하고, 텍스트를 추출한 다음, 청크로 분할합니다.

In [5]:
# PDF 파일 경로 정의
pdf_path = "data/AI_Information.pdf"

# PDF 파일에서 텍스트 추출
extracted_text = extract_text_from_pdf(pdf_path)

# 추출된 텍스트를 1000자 세그먼트로 청킹하고 200자 중첩 허용
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 청크 수 출력
print("텍스트 청크 수:", len(text_chunks))

# 첫 번째 텍스트 청크 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## 텍스트 청크에 대한 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [6]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    지정된 OpenAI 모델을 사용하여 주어진 텍스트에 대한 임베딩을 생성합니다.

    Args:
    text (str): 임베딩을 생성할 입력 텍스트.
    model (str): 임베딩 생성에 사용할 모델. 기본값은 "BAAI/bge-en-icl"입니다.

    Returns:
    dict: 임베딩을 포함하는 OpenAI API의 응답.
    """
    # 지정된 모델을 사용하여 입력 텍스트에 대한 임베딩 생성
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 임베딩을 포함하는 응답 반환

# 텍스트 청크에 대한 임베딩 생성
response = create_embeddings(text_chunks)

## 문맥 인식 시맨틱 검색 구현
더 나은 문맥을 위해 주변 청크를 포함하도록 검색을 수정합니다.
기존의 시맨틱 검색은 가장 유사한 청크만을 반환했지만, 여기서는 가장 유사한 청크와 함께 그 주변의 청크들(예: 이전 청크와 다음 청크)도 함께 반환하여 언어 모델이 답변을 생성할 때 더 넓은 문맥을 참고할 수 있도록 합니다.

In [7]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터.
    vec2 (np.ndarray): 두 번째 벡터.

    Returns:
    float: 두 벡터 간의 코사인 유사도.
    """
    # 두 벡터의 내적을 계산하고 각 벡터의 노름(크기)의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [8]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    가장 관련성 높은 청크와 그 주변 청크를 함께 검색합니다.

    Args:
    query (str): 검색 질의.
    text_chunks (List[str]): 텍스트 청크 목록.
    embeddings (List[dict]): 청크 임베딩 목록.
    k (int): 검색할 관련 청크 수 (여기서는 가장 관련 높은 청크 1개를 찾는데 사용됨).
    context_size (int): 포함할 주변 청크 수 (앞뒤로 각각 context_size 만큼의 청크를 포함).

    Returns:
    List[str]: 문맥 정보를 포함하는 관련 텍스트 청크.
    """
    # 질의를 임베딩 벡터로 변환
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []

    # 질의와 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    for i, chunk_embedding in enumerate(embeddings):
        # 질의 임베딩과 현재 청크 임베딩 간의 코사인 유사도 계산
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        # 인덱스와 유사도 점수를 튜플로 저장
        similarity_scores.append((i, similarity_score))

    # 유사도 점수를 기준으로 청크를 내림차순 정렬 (유사도가 가장 높은 것이 먼저 오도록)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 가장 관련성 높은 청크의 인덱스 가져오기
    top_index = similarity_scores[0][0]

    # 문맥 포함 범위 정의
    # 0보다 작거나 text_chunks 길이를 넘지 않도록 보장
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # 관련 청크와 주변 문맥 청크 반환
    return [text_chunks[i] for i in range(start, end)]

## 문맥 검색을 사용한 질의 실행
이제 문맥 강화 검색을 테스트합니다.

In [9]:
# JSON 파일에서 검증 데이터셋 로드
with open('data/val.json') as f:
    data = json.load(f)

# 데이터셋에서 첫 번째 질문을 질의로 사용
query = data[0]['question']

# 가장 관련성 높은 청크와 문맥을 위한 주변 청크 검색
# 매개변수:
# - query: 검색할 질문
# - text_chunks: PDF에서 추출한 텍스트 청크
# - response.data: 텍스트 청크의 임베딩
# - k=1: 상위 1개 일치 항목 반환 (가장 유사한 청크 1개를 찾음)
# - context_size=1: 문맥을 위해 상위 일치 항목 앞뒤로 각각 1개의 청크 포함
top_chunks = context_enriched_search(query, text_chunks, response.data, k=1, context_size=1)

# 참고용으로 질의 출력
print("질의:", query)
# 검색된 각 청크를 제목과 구분 기호와 함께 출력
for i, chunk in enumerate(top_chunks):
    print(f"문맥 {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:
nt aligns with societal values. Education and awareness campaigns inform the public 
about AI, its impacts, and its potential. 
Chapter 19: AI and Ethics 
Principles of Ethical AI 
Ethical AI principles guide the development and deployment of AI systems to ensure they are fair, 
transparent, accountable, and beneficial to society. Key principles include respect for human 
rights, privacy, non-discrimination, and beneficence. 
 
 
Addressing Bias in AI 
AI systems can inherit and amplify biases present in the data they are trained on, leading to unfair 
or discriminatory outcomes. Addressing bias requires careful data collection, algorithm design, 
and ongoing monitoring and evaluation. 
Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
f

## 검색된 문맥을 사용하여 응답 생성
이제 LLM을 사용하여 응답을 생성합니다.

In [10]:
# AI 어시스턴트에 대한 시스템 프롬프트 정의
system_prompt = "당신은 주어진 문맥에 기반하여 엄격하게 답변하는 AI 어시스턴트입니다. 제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, '답변하기에 충분한 정보가 없습니다.'라고 응답하십시오."

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델로부터 응답을 생성합니다.

    Args:
    system_prompt (str): AI의 행동을 안내하는 시스템 프롬프트.
    user_message (str): 사용자의 메시지 또는 질의.
    model (str): 응답 생성에 사용할 모델. 기본값은 "meta-llama/Llama-3.2-3B-Instruct"입니다. (원문: "meta-llama/Llama-2-7B-chat-hf")

    Returns:
    dict: AI 모델의 응답.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 상위 청크를 기반으로 사용자 프롬프트 생성
user_prompt = "\n".join([f"문맥 {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## AI 응답 평가
AI 응답을 예상 답변과 비교하고 점수를 매깁니다.

In [11]:
# 평가 시스템에 대한 시스템 프롬프트 정의
evaluate_system_prompt = "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. AI 어시스턴트의 응답이 실제 응답과 매우 유사하면 1점을 부여하십시오. 응답이 실제 응답과 비교하여 부정확하거나 만족스럽지 않으면 0점을 부여하십시오. 응답이 실제 응답과 부분적으로 일치하면 0.5점을 부여하십시오."

# 사용자 질의, AI 응답, 실제 응답 및 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt = f"사용자 질의: {query}\nAI 응답:\n{ai_response.choices[0].message.content}\n실제 응답: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 평가 시스템 프롬프트와 평가 프롬프트를 사용하여 평가 응답 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 응답 출력
print(evaluation_response.choices[0].message.content)

Based on the evaluation criteria, I would assign a score of 0.8 to the AI assistant's response.

The response is very close to the true response, and it correctly conveys the main idea of Explainable AI (XAI) and its importance. The AI assistant's response is also well-structured and easy to understand, which is a positive aspect.

However, there are a few minor differences between the AI assistant's response and the true response. The AI assistant's response is slightly more detailed and provides additional points (1-4) that are not present in the true response. Additionally, the AI assistant's response uses more formal language and phrases, such as "In essence," which is not present in the true response.

Despite these minor differences, the AI assistant's response is still very close to the true response, and it effectively conveys the main idea of XAI and its importance. Therefore, I would assign a score of 0.8.
